## Keras performance test

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras_tqdm import TQDMNotebookCallback

import pickle

Using MXNet backend.
/anaconda/envs/py35/lib/python3.5/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:

import cntk
import tensorflow
import theano


print ("Keras Version: ",keras.__version__)
print ("TensorFlow Version: ",tensorflow.__version__)
print ("CNTK Version: ",cntk.__version__)
print ("Theano Version: ",theano.__version__)

/anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/anaconda/envs/py35/lib/python3.5/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "


Keras Version:  1.2.2
TensorFlow Version:  1.3.0
CNTK Version:  2.1
Theano Version:  0.9.0


Using cuDNN version 6021 on context None
Mapped name None to device cuda: Tesla K80 (299E:00:00.0)


In [3]:

batch_size = 128
num_classes = 10
epochs = 20

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [5]:
#output_file='mnist_mlp_y_train.pckl'
#f = open(output_file, 'wb')
#pickle.dump(y_train, f)
#f.close()

#output_file='mnist_mlp_y_test.pckl'
#f = open(output_file, 'wb')
#pickle.dump(y_test, f)
#f.close()



In [4]:
output_file='mnist_mlp_y_train.pckl'
f = open(output_file, 'rb')
y_train = pickle.load(f)
f.close()

output_file='mnist_mlp_y_test.pckl'
f = open(output_file, 'rb')
y_test = pickle.load(f)
f.close()

In [5]:
y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [6]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

#model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'],context=['gpu(0)'])

## Tensorflow

In [11]:
#print ("Tensorflow Version: ",tensorflow.__version__)
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
import timeit
%timeit -r 1 -n 1 history = model.fit(x_train, y_train,batch_size=batch_size,nb_epoch=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Keras Backend:  tensorflow
Epochs:  20


/home/jasmeetsb/notebooks/deep-learning-keras-projects/keras-performance-comparison/keras-performance-cntk-tensorflow/mxnet/keras/backend/tensorflow_backend.py:1960: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))



1min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Test loss: 0.126942871839
Test accuracy: 0.9826


## CNTK

In [5]:
print ("CNTK Version: ",cntk.__version__)
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
%timeit -r 1 -n 1 history = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

CNTK Version:  2.1
Keras Backend:  cntk
Epochs:  20


/anaconda/envs/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input74") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))



55.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Test loss: 0.11012129811
Test accuracy: 0.9822


## Theano

In [ ]:
print ("Theano Version: ",theano.__version__)
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
%timeit -r 1 -n 1 history = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


## MXNet

In [7]:
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
%timeit -r 1 -n 1 history = model.fit(x_train, y_train,batch_size=batch_size,nb_epoch=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Keras Backend:  mxnet
Epochs:  20


/anaconda/envs/py35/lib/python3.5/site-packages/mxnet/module/bucketing_module.py:385: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.0078125). Is this intended?
  force_init=force_init)



29.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Test loss: 0.09823607674
Test accuracy: 0.9841
